In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import torch
import numpy as np
import torch.optim as optim
from torch.utils import data
from sqnet import SQNet
from data import dataGen
from tqdm import tqdm
from jacobian import JacobianReg
import time
import crocoddyl


In [3]:
# Tensor data for training
positions, costs, grad1, _ = dataGen(size = 1000)

In [4]:
# Torch dataloader
dataset = torch.utils.data.TensorDataset(positions,costs, grad1)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 1000) 

In [5]:
# Generate a Neural Net
net = SQNet(input_features = positions.shape[1], 
             output_features = costs.shape[1],
             n_hiddenunits = 16)

In [6]:
# Initialize loss and optimizer
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.95)
lam1 = 0.01


In [7]:
for epoch in tqdm(range(5000)):
    epoch_mse0 = 0.0
    epoch_mse1 = 0.0
    for i, (input, target0, target1) in enumerate(dataloader):  
        
        net.eval()

        output0 = net(input)               
        output1 = net.jacobian(input)

        net.train()
        mse0 = criterion(output0, target0)
        mse1 = criterion(output1, target1)
        

        epoch_mse0 += mse0.item() * len(input)
        epoch_mse1 += mse1.item() * len(input)
        
        loss = mse0 + lam1 * mse1 

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    scheduler.step()
 
    epoch_mse0 /= 3000
    epoch_mse1 /= 3000
    
    epoch_loss = epoch_mse0 + lam1*epoch_mse1 
    print('epoch', epoch,
      'lr', '{:.7f}'.format(optimizer.param_groups[0]['lr']),
      'mse0', '{:.5f}'.format(epoch_mse0),
      'mse1', '{:.5f}'.format(epoch_mse1),
      'Total Loss', '{:.5f}'.format(epoch_loss))
  




  0%|          | 1/5000 [00:00<40:37,  2.05it/s]

epoch 0 lr 0.0010000 mse0 318412.29167 mse1 335365.77083 Total Loss 321765.94938


  0%|          | 2/5000 [00:00<40:24,  2.06it/s]

epoch 1 lr 0.0010000 mse0 318316.16667 mse1 335351.77083 Total Loss 321669.68438


  0%|          | 3/5000 [00:01<40:15,  2.07it/s]

...

epoch 229 lr 0.0010000 mse0 159458.91667 mse1 370100.20833 Total Loss 163159.91875


  5%|▍         | 231/5000 [01:48<38:01,  2.09it/s]

epoch 230 lr 0.0010000 mse0 158667.37500 mse1 370365.50000 Total Loss 162371.03000


  5%|▍         | 231/5000 [01:48<37:21,  2.13it/s]


KeyboardInterrupt: 

In [ ]:
torch.save(net, "sqnet2.pth")